<b>Realizar uma análise sobre a oferta/vitrine das opções de produtos que repondam a distintas buscas no site do mercadolibre.com.ar</b>

<h1>Tarefa 1</h1>

<n>Varrer uma lista de mais de 150 itens ids no serviço público:

https://api.mercadolibre.com/sites/MLA/search?q=chromecast&limit=50#json

<n>Nesse caso particular e somente como exemplo, são resultado para a busca "chromecast", porém deverá eleger outros términos para o experimento que permitam enriquecer uma análise em um hipotético dashboard (ex: Google Home, Apple TV, Amazon Fire TV, e outros para poder comparar dispositivos portáteis, ou até mesmo eleger outros e que você tenha interesse em comparar).</n></n>

<h3>Criação do Dataframe</h3>

In [1]:
# Imports
import requests
import pandas as pd
import functools as a
import csv

In [2]:
item_range = 10
items = ['chromecast', 'Google Home', 'Apple TV', 'Amazon Fire TV']

# Função que varre os itens e retorna suas respectivas informações
def get_info(item):
    return [requests.get(f"https://api.mercadolibre.com/sites/MLA/search?q={item}&limit={item_range}&offset={num}").json()
            for num in range(1, item_range+1)]

# result_list contém os dicionários de informações para cada combinação de item e número de página
result_list = a.reduce(lambda result, item: result + get_info(item), items, [])

In [3]:
# Converter a lista de dicionários em DataFrame
data_MercadoLibre = pd.DataFrame(result_list)

<h3>Verificação dos Dados</h3>

In [4]:
# Cabeçalho do Dataframe criado
data_MercadoLibre.head()

,site_id,country_default_time_zone,query,paging,results,sort,available_sorts,filters,available_filters
0,MLA,GMT-03:00,chromecast,"{'total': 382, 'primary_results': 382, 'offset...","[{'id': 'MLA1150728234', 'title': 'Google Chro...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."
1,MLA,GMT-03:00,chromecast,"{'total': 382, 'primary_results': 382, 'offset...","[{'id': 'MLA1394845862', 'title': 'Xiaomi Mi T...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."
2,MLA,GMT-03:00,chromecast,"{'total': 382, 'primary_results': 382, 'offset...","[{'id': 'MLA1346250874', 'title': 'Chromecast ...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'state', 'name': 'Ubicación', 'type': ..."
3,MLA,GMT-03:00,chromecast,"{'total': 382, 'primary_results': 382, 'offset...","[{'id': 'MLA1200056325', 'title': 'Google Chro...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."
4,MLA,GMT-03:00,chromecast,"{'total': 382, 'primary_results': 382, 'offset...","[{'id': 'MLA1400348604', 'title': 'Xiaomi Stic...","{'id': 'relevance', 'name': 'Más relevantes'}","[{'id': 'price_asc', 'name': 'Menor precio'}, ...","[{'id': 'category', 'name': 'Categorías', 'typ...","[{'id': 'official_store', 'name': 'Tiendas ofi..."


In [5]:
# Exemplo do dado desejado
print(data_MercadoLibre['results'][0][1]['id'])

MLA1394845862


In [6]:
# Tipo dos Dados
data_MercadoLibre.dtypes

site_id                      object
country_default_time_zone    object
query                        object
paging                       object
results                      object
sort                         object
available_sorts              object
filters                      object
available_filters            object
dtype: object

In [11]:
# Verifica quais os possíveis resultados de busca
data_MercadoLibre['query'].unique()

array(['chromecast', 'Google Home', 'Apple TV', 'Amazon Fire TV'],
      dtype=object)

<h1>Tarefa 2</h1>

<n>Para cada resultado, realizar o correspondente GET por item_id no recurso publico:</n>

https://api.mercadolibre.com/items/{Item_Id}

In [12]:
# Obtém as informações do item com base no Item_id.
def get_item_info(item_id):
    url = f"https://api.mercadolibre.com/items/{item_id}"
    response = requests.get(url)

    if response.status_code == 200:
        item_info = response.json()
        return item_info
    else:
        return None

items_mercado_libre = [get_item_info(result['id']) for result in result_list[0]['results']]

<h1>Tarefa 3</h1>

<n>Escrever os resultados em um arquivo plano delimitado por vírgulas, desnormalizando o JSON obtido no passo anterior, em quantos campos sejam necessários para guardar as variáveis que te interesse modelar.</n>

In [15]:
features = ['site_id', 'title', 'subtitle', 'seller_id', 'category_id', 'official_store_id', 'price', 'base_price'
            , 'original_price', 'currency_id' 'initial_quantity', 'available_quantity', 'sold_quantity', 'sale_terms',
               'buying_mode']

# Caminho do arquivo de saída CSV
result_csv_file = '\projetos\mercado_libre_items.csv'

with open(result_csv_file, 'w', newline='', encoding='utf-8') as result_file:
    writer = csv.writer(result_file)
    
    # Escreve o cabeçalho com os nomes dos campos
    writer.writerow(features)
    
    # Itera sobre os itens e escreve as informações nos campos correspondentes
    for item in items_mercado_libre:
        row = []
        
        # Extrai as informações desejadas do item
        for feature in features:
            # Verifica se o campo está presente no item
            if feature in item:
                row.append(item[feature])
            else:
                row.append('')
        
        # Escreve a linha no arquivo CSV
        writer.writerow(row)

In [14]:
items_mercado_libre

[{'id': 'MLA1150728234',
  'site_id': 'MLA',
  'title': 'Google Chromecast With Google Tv De Voz 4k 8gb Snow Con 2gb De Memoria Ram',
  'subtitle': None,
  'seller_id': 701293922,
  'category_id': 'MLA352001',
  'official_store_id': None,
  'price': 37621,
  'base_price': 37621,
  'original_price': None,
  'currency_id': 'ARS',
  'initial_quantity': 61,
  'available_quantity': 1,
  'sold_quantity': 50,
  'sale_terms': [{'id': 'INVOICE',
    'name': 'Facturación',
    'value_id': '6891885',
    'value_name': 'Factura A',
    'value_struct': None,
    'values': [{'id': '6891885', 'name': 'Factura A', 'struct': None}],
    'value_type': 'list'},
   {'id': 'WARRANTY_TYPE',
    'name': 'Tipo de garantía',
    'value_id': '2230280',
    'value_name': 'Garantía del vendedor',
    'value_struct': None,
    'values': [{'id': '2230280',
      'name': 'Garantía del vendedor',
      'struct': None}],
    'value_type': 'list'},
   {'id': 'WARRANTY_TIME',
    'name': 'Tiempo de garantía',
    'value